In [1]:
import os
import copy
import time
import pandas as pd
from torchvision.io import read_image
from torchvision.datasets.folder import default_loader
from torchvision.datasets.utils import download_url
from torch.utils.data import Dataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torchvision.models import vgg16
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.utils.prune as prune
from heapq import nsmallest
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

c:\Users\Victor\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
!nvidia-smi

Thu Nov 10 17:43:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:04:00.0  On |                  N/A |
| 43%   30C    P0    26W /  95W |    937MiB /  2048MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40 24GB     TCC   | 00000000:2B:00.0 Off |                  N/A |
| N/A   

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class Cub2011(Dataset):
    base_folder = 'CUB_200_2011/images'
    url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1'
    filename = 'CUB_200_2011.tgz'
    tgz_md5 = '97eceeb196236b17998738112f37df78'

    def __init__(self, root, train=True, transform=None, loader=default_loader, download=True):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.loader = default_loader
        self.train = train

        if download:
            self._download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

    def _load_metadata(self):
        images = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'images.txt'), sep=' ',
                             names=['img_id', 'filepath'])
        image_class_labels = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'image_class_labels.txt'),
                                         sep=' ', names=['img_id', 'target'])
        train_test_split = pd.read_csv(os.path.join(self.root, 'CUB_200_2011', 'train_test_split.txt'),
                                       sep=' ', names=['img_id', 'is_training_img'])

        data = images.merge(image_class_labels, on='img_id')
        self.data = data.merge(train_test_split, on='img_id')

        if self.train:
            self.data = self.data[self.data.is_training_img == 1]
        else:
            self.data = self.data[self.data.is_training_img == 0]

    def _check_integrity(self):
        try:
            self._load_metadata()
        except Exception:
            return False

        for index, row in self.data.iterrows():
            filepath = os.path.join(self.root, self.base_folder, row.filepath)
            if not os.path.isfile(filepath):
                print(filepath)
                return False
        return True

    def _download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        download_url(self.url, self.root, self.filename, self.tgz_md5)

        with tarfile.open(os.path.join(self.root, self.filename), "r:gz") as tar:
            tar.extractall(path=self.root)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        path = os.path.join(self.root, self.base_folder, sample.filepath)
        target = sample.target - 1 
        img = self.loader(path)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

In [6]:
transform = T.Compose([
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
train_ds = Cub2011('.', train=True, transform = transform)
val_ds = Cub2011('.s', train=False, transform = transform)

ds = {'train': DataLoader(train_ds, batch_size = 64, shuffle=True),
      'val': DataLoader(val_ds, batch_size = 64, shuffle=False)}


ds_sizes = {'train': len(train_ds),
      'val': len(val_ds)}

  4%|▍         | 43614208/1150585339 [00:04<01:51, 9941270.50it/s] 


KeyboardInterrupt: 

In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, nclas, dataset_sizes, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_bal_acc = 0.0

    val_bal_acc = []
    val_acc = []
    val_loss = []

    train_bal_acc = []
    train_acc = []
    train_loss = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            CF = np.zeros((nclas,nclas)) # Confusion matrix

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                for i in range(len(labels.data)):
                    CF[labels.data[i]][preds[i]] +=1

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            recalli = 0
            for i in range(nclas):
                TP = CF[i][i]
                FN = 0
                for j in range(nclas):
                    if i!=j:
                        FN+=CF[i][j]
                if (TP+FN) !=0:
                    recalli+= TP/(TP+FN)
            epoch_bal_acc = recalli/nclas

            if phase == 'val':
                val_bal_acc.append(epoch_bal_acc)
                val_acc.append(epoch_acc)
                val_loss.append(epoch_loss)
            else:
                train_bal_acc.append(epoch_bal_acc)
                train_acc.append(epoch_acc)
                train_loss.append(epoch_loss)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Balanced Acc: {epoch_bal_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_bal_acc > best_bal_acc:
                best_acc = epoch_acc
                best_bal_acc = epoch_bal_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    #print(f'Best val Acc: {best_acc:4f}')
    print(f'Best val Balanced Acc: {best_bal_acc:4f}')

    print('Validation:')
    print('Val_bal_acc:', val_bal_acc)
    print('Val_acc:', val_acc)
    print('Val_loss:', val_loss)

    print('Training:')
    print('Train_bal_acc:', train_bal_acc)
    print('Train_acc:', train_acc)
    print('Train_loss:', train_loss)

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Finetuning the model

In [9]:
model_ft = models.vgg16(weights='IMAGENET1K_V1')
model_ft.classifier[6] = nn.Linear(4096, 200)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [13]:
model_conv = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, ds, 200,ds_sizes, num_epochs=25)

Epoch 0/24
----------
train Loss: 4.9987 Acc: 0.0410 Balanced Acc: 0.0410
val Loss: 3.7867 Acc: 0.1866 Balanced Acc: 0.1860

Epoch 1/24
----------
train Loss: 3.3310 Acc: 0.2296 Balanced Acc: 0.2296
val Loss: 2.5529 Acc: 0.3781 Balanced Acc: 0.3791

Epoch 2/24
----------
train Loss: 2.5370 Acc: 0.3642 Balanced Acc: 0.3641
val Loss: 2.1254 Acc: 0.4746 Balanced Acc: 0.4785

Epoch 3/24
----------
train Loss: 2.1561 Acc: 0.4548 Balanced Acc: 0.4547
val Loss: 1.9841 Acc: 0.4858 Balanced Acc: 0.4883

Epoch 4/24
----------
train Loss: 1.9385 Acc: 0.4910 Balanced Acc: 0.4909
val Loss: 1.8575 Acc: 0.5233 Balanced Acc: 0.5272

Epoch 5/24
----------
train Loss: 1.7638 Acc: 0.5354 Balanced Acc: 0.5353
val Loss: 1.7252 Acc: 0.5426 Balanced Acc: 0.5457

Epoch 6/24
----------
train Loss: 1.6557 Acc: 0.5677 Balanced Acc: 0.5677
val Loss: 1.7084 Acc: 0.5573 Balanced Acc: 0.5606

Epoch 7/24
----------
train Loss: 1.3973 Acc: 0.6275 Balanced Acc: 0.6274
val Loss: 1.5898 Acc: 0.5939 Balanced Acc: 0.5960



In [17]:
torch.save(model_conv, 'fine_tuning_model.pth')